In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

# Static Configurations

> In this module, we configure stuff.

In [ ]:
#| default_exp configuration

## Imports

In [ ]:
#| export

from pathlib import Path
import thucy
from dotenv import set_key, load_dotenv, dotenv_values
from pydantic import BaseModel

## Configuration Class

The `Config` class manages all configuration through a combination of default values and environment variables loaded from `.env`.

In [ ]:
#| export

class Config:
    """Global configuration for the thucy project"""

    DEFAULTS = {
        "EXPERTS_MODEL": "gpt-5-mini",
        "LEAD_MODEL": "gpt-5",
        "LEAD_MAX_TURNS": "40",
        "SQL_EXPERT_MAX_TURNS": "30",
        "SCHEMA_EXPERT_MAX_TURNS": "30",
        "DATA_EXPERT_MAX_TURNS": "30",
        "GENAI_SERVER_URL": "http://127.0.0.1:5000",
    }
    
    def __init__(self):
        # Path Configs
        self.project_root = Path(thucy.__file__).parent.parent
        self.expertiments_dir = self.project_root / "experiments"
        self.results_dir = self.expertiments_dir / "results"  
        self.logs_dir = self.expertiments_dir / "logs" 

        self.env_path = self.project_root / ".env"
        self.env_path.touch(exist_ok=True)

        # Load environment variables
        self._load_env()

    def _load_env(self):
        """Load everything in DEFAULTS + everything in the .env file."""
        # Load into the environment to make sure
        load_dotenv(self.env_path, override=True)
        env = dotenv_values(self.env_path)

        for key, val in env.items():
            if val.isdigit(): setattr(self, key.lower(), int(val))
            else: setattr(self, key.lower(), val)
    
    def restore_defaults(self):
        for key, val in self.DEFAULTS.items():
            self.set_env_var(key, str(val))

    def set_env_var(self, key, value):
        """Set or update a variable and persist to .env."""
        set_key(self.env_path, key, str(value))
        self._load_env()

    def show_config(self):
        env = dotenv_values(self.env_path)
        for key, val in env.items():
            print(f"{key}={val}")

### Singleton Instance

We create a single global `config` instance that can be imported throughout the project.

In [ ]:
#| exports

config = Config()

In [ ]:
config.restore_defaults()

In [ ]:
config.show_config()

OPENAI_API_KEY=sk-xxyy
EXPERTS_MODEL=gpt-5-mini
LEAD_MODEL=gpt-5
SQL_EXPERT_MAX_TURNS=30
SCHEMA_EXPERT_MAX_TURNS=30
DATA_EXPERT_MAX_TURNS=30
GENAI_SERVER_URL=http://127.0.0.1:5000
LEAD_MAX_TURNS=40


### Usage Examples

The following cells demonstrate how to set and retrieve environment variables using the config system.

Check if variable exists before setting it:

In [ ]:
import os

In [ ]:
os.getenv("SQL_EXPERT_MAX_TURNS")

'30'

Set the variable and verify it's saved:

In [ ]:
config.set_env_var('SQL_EXPERT_MAX_TURNS', 5)

In [ ]:
os.getenv("SQL_EXPERT_MAX_TURNS")

'5'

In [ ]:
config.sql_expert_max_turns

5

In [ ]:
config.set_env_var('EXPERTS_MODEL', 'gpt-5.1')

In [ ]:
os.getenv("EXPERTS_MODEL")

'gpt-5.1'

Access via config object (lowercase attribute):

In [ ]:
config.experts_model

'gpt-5.1'

Restore to original default values:

In [ ]:
config.restore_defaults()

In [ ]:
#| notest

config.show_config()

OPENAI_API_KEY=sk-xxyy
EXPERTS_MODEL=gpt-5-mini
LEAD_MODEL=gpt-5
SQL_EXPERT_MAX_TURNS=30
SCHEMA_EXPERT_MAX_TURNS=30
DATA_EXPERT_MAX_TURNS=30
GENAI_SERVER_URL=http://127.0.0.1:5000
LEAD_MAX_TURNS=40


In [ ]:
config.lead_max_turns

40

In [ ]:
#| export 

def append_to_file(filename: str,  # filename to append to
                   data: BaseModel,  # a Pydantic BaseModel instance with attributes to log
                   general_tag: str  # a general tag to wrap around the data
                   ):
    """ Append the attributes of a Pydantic BaseModel instance to a file within a general tag."""
    with open(config.results_dir / filename, "a", encoding="utf-8") as f:

        f.write(f"\n<{general_tag}>\n")

        for field, value in data.model_dump().items():
            f.write(f"<{field}>\n{value}\n</{field}>\n")

        f.write(f"</{general_tag}>\n")

In [ ]:
class TestBucket(BaseModel):
    name: str = "TestBucket"
    size: int = 10
    region: str = "us-west-1"

append_to_file("test_log.txt", data=TestBucket(), general_tag="CONFIGURATION")

In [ ]:
with open(config.results_dir / "test_log.txt", "r", encoding="utf-8") as f:
    print(f.read())


<CONFIGURATION>
<name>
TestBucket
</name>
<size>
10
</size>
<region>
us-west-1
</region>
</CONFIGURATION>



In [ ]:
os.remove(config.results_dir / "test_log.txt")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()